In [4]:
import cv2
import json
import os
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers.schedules import ExponentialDecay
import numpy as np


In [5]:
def extract_video_features(video_path):
    cap = cv2.VideoCapture(video_path)
    #check if video opened successfully
    if not cap.isOpened():
        raise ValueError(f"Could not open video file: {video_path}")
    #extract video resolution and fps
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    return cap, width, height, fps


In [6]:
'''
Script to take a video and convert it to downsample size and save it
The labels in the test files are also downsized
'''
def process_video_and_labels(video_path, input_json_path, output_video_path, output_json_path, target_size=(320, 220)):
    #check if downsampled video exists
    #get original video properties
    cap, original_width, original_height, fps = extract_video_features(video_path)
    # get scaling from original to target size
    scale_x = target_size[0] / original_width
    scale_y = target_size[1] / original_height
    if not os.path.exists(output_video_path):
        # Create video writer
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_video_path, fourcc, fps, target_size)
        while True:
            #extract video frames
            ret, frame = cap.read()
            if not ret:
                break
            #resize video frames and write to output video
            resized = cv2.resize(frame, target_size)  # target_size is (width, height)
            out.write(resized)

        out.release()
        print(f"Saved resized video to: {output_video_path}")
    cap.release()
    #Check if dowmsampled labels exist
    if not os.path.exists(output_json_path):
        with open(input_json_path, 'r') as f:
            data = json.load(f)

        scaled_data = {
            frame: None if coords is None else {
                "x": coords["x"] * scale_x,
                "y": coords["y"] * scale_y
            }
            for frame, coords in data.items()
        }

        with open(output_json_path, 'w') as f:
            json.dump(scaled_data, f, indent=4)
        print(f"Saved scaled labels to: {output_json_path}")


In [7]:
def extract_specific_frames(video_path, frame_indices):
    cap = cv2.VideoCapture(video_path)
    extracted_frames = {}

    if not cap.isOpened():
        print("Error: Cannot open video.")
        return extracted_frames

    for index in frame_indices:
        cap.set(cv2.CAP_PROP_POS_FRAMES, index)
        success, frame = cap.read()
        if success:
            extracted_frames[index] = frame
        else:
            print(f"Warning: Could not read frame {index}")

    cap.release()
    return extracted_frames


In [8]:
model_path = "../models/tracknet_pre_model.keras"
model = tf.keras.models.load_model(model_path, compile = False)
lr_scheduler = ExponentialDecay(
        initial_learning_rate=0.001,
        decay_steps=100,
        decay_rate=0.96)
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_scheduler)
input_shape=(220, 320, 3)
inputs = layers.Input(shape=input_shape)
outputs = model(inputs)
model = tf.keras.Model(inputs, outputs)
model.compile(optimizer=optimizer, loss='mean_squared_error', metrics = ['accuracy', "f1_score"])

In [ ]:
test_folder = "../data/test/"
mse_s = []
labels = []
for i in range(1,8):
    video_file = test_folder + f"test_{i}.mp4"
    output_video_file = test_folder +f"test_{i}_down.mp4"
    ball_markup = test_folder + f"test_{i}/ball_markup.json"
    output_ball_mark = test_folder + f"test_{i}/ball_markup_down.json"
    process_video_and_labels(video_file, ball_markup, output_video_file, output_ball_mark)

    with open(output_ball_mark, 'r') as f:
        events_data = json.load(f)
    frame_nums = list(map(int, events_data.keys()))
    frames = extract_specific_frames(output_video_file, frame_nums)


    # print(cropped_frames)
    event_data = []
    for event in events_data.values():
        event_data.append(event)

    out_frames = []
    for frame in frames.values():
        out_frames.append(frame)
    frames = np.array(out_frames)
    outputs= np.array([[point['x'], point['y']] for point in event_data])
    print(frames.shape)
    print(outputs.shape)
    results = model.predict(frames)
    print(results.shape)
    mse_fn=tf.keras.losses.MeanSquaredError()
    mse = mse_fn(outputs, results)
    print("MSE Score:", mse)
    mse_s.append(mse)
    labels.append(f"Video {i}")



  



(887, 220, 320, 3)
(887, 2)


In [ ]:
import matplotlib.pyplot as plt

# Create the bar chart
plt.figure(figsize=(10, 6))
plt.bar(labels, mse_s, color='skyblue')

# Add title and labels
plt.title('Ball Events Accuracy')
plt.xlabel('Video Number')
plt.ylabel('Accuracy')

# Optionally add grid and display
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Show the plot
plt.show()
